In [1]:
# Install library
# !pip install beautifulsoup4
# !pip install webdriver-manager
# !pip install lxml
# !pip install selenium
# !pip install numpy
# !pip install pandas

In [2]:
# Import library  
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime as dt
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [3]:
# Webdriver Configuration and Url 
url= 'https://yankes.kemkes.go.id/app/siranap/rumah_sakit?jenis=1&propinsi=31prop&kabkota'

s = Service("C:/chromedriver/chromedriver")
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get(url)
time.sleep(0.5)

In [4]:
# Cari List Kota
content = driver.page_source
soup = BeautifulSoup(content, 'lxml')

list_kota = soup.find('select', {'id': 'kabkota'})
kota = list_kota.findAll('option')

In [5]:
# Menampung Data

data_provinsi = []
data_kota = []
data_nama_rs = []
data_kategori_tempat_tidur = []
data_no_tlpn = []
data_alamat_rs = []
data_tempat_tidur = []
data_tempat_tidur_kosong = []
data_tempat_tidur_antri = []
data_tanggal_update = []

for i in kota:
    value = i['value']
    nama_provinsi = 'DKI Jakarta'
    nama_kota = i.text

    if not value or str(value) == "3101":
        continue

    url = 'https://yankes.kemkes.go.id/app/siranap/rumah_sakit?jenis=1&propinsi=31prop&kabkota={id_kota}'.format(id_kota=value)
    driver.get(url)
    time.sleep(0.5)
    
    content = driver.page_source
    soup = BeautifulSoup(content, 'lxml')

    list_card = soup.find_all("div", {"class": "cardRS"})
    n_card = len(list_card)
    
    for a in range(1, n_card):
        card = list_card[a]
        nama_rs = card.find('h5').text

        driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[{}]/div/div[2]/div/a/i'.format(a)).click()

        content = driver.page_source
        soup = BeautifulSoup(content, 'lxml')

        div = soup.findAll('div', class_='container pt-3 pb-1') 
        z = div[0].find('p',class_='mb-0')
        zz = z.findAll('small')

        alamat = zz[0].text.strip()
        no_tlp = zz[1].text.strip() 

        detail_list_card = soup.find_all("div", {"class": "card"})

        for detail_card in detail_list_card:
            detail_kategori = detail_card.find('p',class_='mb-0').text.strip().split("\n")
            kategori = detail_kategori[0]
            tanggal_update = detail_kategori[1].split()[1]
            data = detail_card.find_all("div", {"class": "text-center pt-1 pb-1"})

            tempat_tidur = 0
            kosong = 0
            antrian = 0

            for bed_detail in data:
                jenis = bed_detail.find('b').text.lower()
                value = int(bed_detail.find("div", {"style": "font-size:20px"}).text)
                if jenis == "tempat tidur":
                    tempat_tidur = value
                if jenis == "kosong":
                    kosong = value
                if jenis == "antrian":
                    antrian = value
            
            
            data_provinsi.append(nama_provinsi)
            data_kota.append(nama_kota)
            data_nama_rs.append(nama_rs)
            data_kategori_tempat_tidur.append(kategori)
            data_no_tlpn.append(no_tlp)
            data_alamat_rs.append(alamat)
            data_tempat_tidur.append(tempat_tidur)
            data_tempat_tidur_kosong.append(kosong)
            data_tempat_tidur_antri.append(antrian)
            data_tanggal_update.append(tanggal_update)
            
            
        driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/span').click()

clean = []
for b in range(len(data_tanggal_update)):
    clean.append(data_tanggal_update[b].replace('Update',''))
        
df = pd.DataFrame()
        
df["provinsi"] = data_provinsi
df["kota"] = data_kota
df["nama_rumah_sakit"] = data_nama_rs
df["kategori_tempat_tidur"] = data_kategori_tempat_tidur
df["no_tlp"] = data_no_tlpn
df["alamat"] = data_alamat_rs
df["tempat_tidur"] = data_tempat_tidur
df["kosong"] = data_tempat_tidur_kosong
df["antri"] = data_tempat_tidur_antri
df["tanggal_update"] = clean

df.to_excel("test.xlsx",sheet_name='data', index=True)
driver.quit()

C:\Users\Wahyu\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:427: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   provinsi               760 non-null    object
 1   kota                   760 non-null    object
 2   nama_rumah_sakit       760 non-null    object
 3   kategori_tempat_tidur  760 non-null    object
 4   no_tlp                 760 non-null    object
 5   alamat                 760 non-null    object
 6   tempat_tidur           760 non-null    int64 
 7   kosong                 760 non-null    int64 
 8   antri                  760 non-null    int64 
 9   tanggal_update         760 non-null    object
dtypes: int64(3), object(7)
memory usage: 59.5+ KB


In [6]:
df.describe()

,tempat_tidur,kosong,antri
count,760.000000,760.000000,760.000000
mean,8.969737,5.130263,0.030263
std,23.438488,13.580391,0.565014
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,6.000000,3.000000,0.000000
max,210.000000,122.000000,15.000000


In [9]:
df.shape

(752, 10)

In [14]:
df.columns

Index(['provinsi', 'kota', 'nama_rumah_sakit', 'kategori_tempat_tidur',
       'no_tlp', 'alamat', 'tempat_tidur', 'kosong', 'antri',
       'tanggal_update'],
      dtype='object')

In [16]:
df

,provinsi,kota,nama_rumah_sakit,kategori_tempat_tidur,no_tlp,alamat,tempat_tidur,kosong,antri,tanggal_update
0,DKI Jakarta,Kota Jakarta Selatan,RS Umum Daerah Mampang Prapatan,IGD Khusus Covid,02127939067,Jl. Jatayu RW. 001 RT. 012 Kelurahan Kebayoran...,5,5,0,04-02-2022
1,DKI Jakarta,Kota Jakarta Selatan,RS Umum Daerah Mampang Prapatan,PICU Khusus Covid,02127939067,Jl. Jatayu RW. 001 RT. 012 Kelurahan Kebayoran...,0,0,0,04-02-2022
2,DKI Jakarta,Kota Jakarta Selatan,RS Umum Daerah Mampang Prapatan,NICU Khusus Covid,02127939067,Jl. Jatayu RW. 001 RT. 012 Kelurahan Kebayoran...,0,0,0,04-02-2022
3,DKI Jakarta,Kota Jakarta Selatan,RS Umum Daerah Mampang Prapatan,Isolasi Tanpa Tekanan Negatif,02127939067,Jl. Jatayu RW. 001 RT. 012 Kelurahan Kebayoran...,0,0,0,04-02-2022
4,DKI Jakarta,Kota Jakarta Selatan,RS Umum Daerah Mampang Prapatan,Isolasi Tekanan Negatif,02127939067,Jl. Jatayu RW. 001 RT. 012 Kelurahan Kebayoran...,50,28,0,04-02-2022
...,...,...,...,...,...,...,...,...,...,...
747,DKI Jakarta,Kota Jakarta Utara,RS Umum Daerah Koja,Isolasi Tekanan Negatif,0818886583,Jl. Agung Utara Raya No. 1 Jakut,0,0,0,04-02-2022
748,DKI Jakarta,Kota Jakarta Utara,RS Umum Daerah Koja,ICU Tanpa Tekanan Negatif Tanpa Ventilator,0818886583,Jl. Agung Utara Raya No. 1 Jakut,0,0,0,04-02-2022
749,DKI Jakarta,Kota Jakarta Utara,RS Umum Daerah Koja,ICU Tanpa Tekanan Negatif Dengan Ventilator,0818886583,Jl. Agung Utara Raya No. 1 Jakut,0,0,0,04-02-2022
750,DKI Jakarta,Kota Jakarta Utara,RS Umum Daerah Koja,ICU Tekanan Negatif tanpa Ventilator,0818886583,Jl. Agung Utara Raya No. 1 Jakut,0,0,0,04-02-2022
